# Machine Translation with Transformer
Terjemahan mesin (machine translation) merupakan salah satu tonggak penting dalam perkembangan Natural Language Processing (NLP). Model Transformer, yang diperkenalkan oleh Vaswani dkk. (2017), menjadi revolusi besar karena mampu menggantikan arsitektur berbasis RNN dan CNN dengan mekanisme attention yang lebih efisien.

Dalam notebook ini, akan dibangun sebuah model Transformer dari nol untuk melakukan penerjemahan otomatis dari bahasa Inggris ke bahasa Spanyol. Pendekatan ini tidak menggunakan model pra-latih, melainkan mengimplementasikan komponen inti Transformer, mulai dari positional encoding, multi-head attention, encoder-decoder layer, hingga proses training.

Fokus utama bukan hanya pada hasil terjemahan, melainkan juga pada kemampuan implementasi dan pemahaman mekanisme internal Transformer. Hal ini menjadi relevan karena Transformer adalah fondasi dari berbagai model modern seperti BERT, GPT, dan ChatGPT, sehingga menguasai arsitektur dasarnya menjadi bekal penting bagi seorang Machine Learning Engineer maupun Data Scientist.

The dataset source is www.manythings.org/anki

## Sequence-to-sequence learning

### Import Library

In [2]:
import os
import re
import pathlib
import random
import string
import zipfile
import numpy as np
import tensorflow as tf
from nltk.translate.bleu_score import sentence_bleu
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import TextVectorization
from tensorflow.keras.layers import Bidirectional,GRU,LSTM,Embedding
from tensorflow.keras.layers import Dense,MultiHeadAttention,LayerNormalization,Embedding,Dropout,Layer
from tensorflow.keras import Sequential,Input
from tensorflow.keras.callbacks import ModelCheckpoint

### Download dan Ekstrak Dataset

#### Download ZIP (tanpa ekstrak otomatis)

In [3]:
path_to_zip = keras.utils.get_file(
    "spa-eng.zip",
    origin="http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip",
    extract=False)

#### Ekstraksi manual

In [4]:
with zipfile.ZipFile(path_to_zip, 'r') as zip_ref:
    zip_ref.extractall(pathlib.Path(path_to_zip).parent)

text_file = pathlib.Path(path_to_zip).parent / "spa-eng" / "spa.txt"

#### Melihat semua file hasil ekstraksi

In [5]:
with open(text_file, encoding="utf-8") as f:
    lines = f.read().split("\n")[:-1]

### Persiapkan Pasangan Teks

Setiap baris berisi kalimat bahasa Inggris dan kalimat bahasa Spanyol yang sesuai. Kalimat bahasa Inggris adalah urutan sumber dan kalimat bahasa Spanyol adalah urutan target. Kami menambahkan token "[start]" di depan dan kami menambahkan token "[end]" di belakang kalimat bahasa Spanyol.

In [6]:
# Start dan end menandakan mulai diubah dan berhentinya translate kata/frasa tersebut

text_pairs = []
for line in lines:
    english, spanish = line.split("\t")
    spanish = "[start] " + spanish + " [end]"
    text_pairs.append((english, spanish))

In [7]:
print("Number of pairs:", len(text_pairs))
print("Example text:", text_pairs[2900])

Number of pairs: 118964
Example text: ("I'll be fine.", '[start] Estaré bien. [end]')


Printing pasangan teks acak Inggris - Spanyol

In [8]:
import random
print(random.choice(text_pairs))

('Where can I buy envelopes?', '[start] ¿Dónde puedo comprar sobres? [end]')


### Split Data

- train_pairs: 70% awal data
- val_pairs: 15% berikutnya
- test_pairs: 15% terakhir
- 70% training: cukup besar untuk belajar pola.
- 15% validation: cukup untuk tuning hyperparameter.
- 15% test: cukup untuk mengukur performa generalisasi model.

#### Menentukan jumlah data validasi 15% dari total data

In [9]:
random.shuffle(text_pairs)

num_val_samples = int(0.15 * len(text_pairs))

#### Mengambil sisa data untuk training, setelah menyisihkan 15% untuk validasi dan 15% untuk testing.

In [10]:
num_train_samples = len(text_pairs) - 2 * num_val_samples
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples:num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples:]

In [11]:
print("Number of train data:", len(train_pairs))
print("Number of validation data:", len(val_pairs))
print("Number of test data:", len(test_pairs))

Number of train data: 83276
Number of validation data: 17844
Number of test data: 17844


### Preprocessing dan Vectorization

* string.punctuation adalah built-in Python yang berisi semua karakter tanda baca umum
* Ditambah + "¿" karena simbol “¿” (inverted question mark) adalah tanda tanya pembuka dalam bahasa Spanyol (contoh: ¿Cómo estás?).
* Membuat daftar semua karakter yang dianggap "tanda baca yang ingin dihapus atau dibersihkan dari teks".

In [12]:
def custom_standardization(input_string):
    lowercase = tf.strings.lower(input_string)
    return tf.strings.regex_replace(lowercase, f"[{re.escape(string.punctuation)}¿]", "")

In [13]:
vocab_size = 15000

Artinya: hanya 15.000 kata yang paling sering muncul yang akan digunakan dalam kamus (vocab). Selanjutnya, menentukan panjang maksimal urutan token setelah teks di-tokenisasi.

In [14]:
sequence_length = 20 

* Membuat vektorisasi untuk input (bahasa Inggris)
* Digunakan untuk encoder, yang hanya perlu urutan teks biasa (tanpa token khusus seperti [start] / [end]).

In [15]:
source_vectorization = layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length,
)

* Membuat vektorisasi untuk target (bahasa Spanyol)
* output_sequence_length=20: Karena target punya token [start] dan [end], maka panjang target lebih panjang 1 token dari input
* standardize=custom_standardization: Menggunakan fungsi pembersih teks kustom (misal: hilangkan tanda baca tertentu tapi pertahankan [start] dan [end]).

In [16]:
target_vectorization = layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length + 1,
    standardize=custom_standardization,
)

#### Ekstrak semua kalimat bahasa Inggris dari pasangan data pelatihan.

In [17]:
train_english_texts = [pair[0] for pair in train_pairs]

#### Ekstrak semua kalimat bahasa Spanyol (dengan [start] dan [end])

In [18]:
train_spanish_texts = [pair[1] for pair in train_pairs]

#### Melatih TextVectorization layer untuk membangun vocabulary berdasarkan teks pelatihan.
`.adapt()` penting agar vektorisasi mengenali kata-kata umum dan menyusun vocab secara efisien.

In [19]:
source_vectorization.adapt(train_english_texts)
target_vectorization.adapt(train_spanish_texts)

### Format Dataset

In [20]:
batch_size = 64

Artinya: Model akan memproses 64 pasangan kalimat (Inggris–Spanyol) dalam satu langkah.

In [21]:
def format_dataset(eng, spa):
    eng = source_vectorization(eng) # Tokenisasi input (bahasa Inggris)
    spa = target_vectorization(spa) # Tokenisasi target (bahasa Spanyol)
    return ({"english": eng, "spanish": spa[:, :-1]}, spa[:, 1:])
    # Semua token target KECUALI yang terakhir atau KECUALI pertama

def make_dataset(pairs):
    # Pisahkan pasangan (eng, spa) jadi 2 list
    eng_texts, spa_texts = zip(*pairs) 
    
    # Buat tf.data.Dataset dari pasangan teks
    dataset = tf.data.Dataset.from_tensor_slices((list(eng_texts), list(spa_texts)))
    dataset = dataset.batch(batch_size) # Bagi data jadi batch ukuran 64
    dataset = dataset.map(format_dataset, num_parallel_calls=tf.data.AUTOTUNE) # Format isi batch dengan format_dataset
    return dataset.shuffle(2048).prefetch(tf.data.AUTOTUNE).cache() # Simpan data hasil transformasi di memori untuk efisiensi

In [22]:
train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)

In [23]:
train_ds

<CacheDataset element_spec=({'english': TensorSpec(shape=(None, 20), dtype=tf.int64, name=None), 'spanish': TensorSpec(shape=(None, 20), dtype=tf.int64, name=None)}, TensorSpec(shape=(None, 20), dtype=tf.int64, name=None))>

In [24]:
val_ds

<CacheDataset element_spec=({'english': TensorSpec(shape=(None, 20), dtype=tf.int64, name=None), 'spanish': TensorSpec(shape=(None, 20), dtype=tf.int64, name=None)}, TensorSpec(shape=(None, 20), dtype=tf.int64, name=None))>

In [25]:
for inputs, targets in train_ds.take(1):
    print(f"english {inputs['english']},\n\n\n inputs['english'].shape: {inputs['english'].shape}")
    print(f"spanish {inputs['spanish']},\n\n\n inputs['spanish'].shape: {inputs['spanish'].shape}")
    print(f"targets {targets}, \n\n\n targets.shape: {targets.shape}")

english [[1439   13 1143 ...    0    0    0]
 [   3  214    7 ...    0    0    0]
 [  87   51    5 ...    0    0    0]
 ...
 [  15    5   17 ...    0    0    0]
 [   6  319  493 ...    0    0    0]
 [   6 1186    7 ...    0    0    0]],


 inputs['english'].shape: (64, 20)
spanish [[   2   35  287 ...    0    0    0]
 [   2  434   18 ...    0    0    0]
 [   2   83  523 ...    0    0    0]
 ...
 [   2   94   18 ...    0    0    0]
 [   2    8 7995 ...    0    0    0]
 [   2    8   26 ...    0    0    0]],


 inputs['spanish'].shape: (64, 20)
targets [[  35  287  113 ...    0    0    0]
 [ 434   18    1 ...    0    0    0]
 [  83  523   28 ...    0    0    0]
 ...
 [  94   18  531 ...    0    0    0]
 [   8 7995   84 ...    0    0    0]
 [   8   26 1367 ...    0    0    0]], 


 targets.shape: (64, 20)


## Sequence-to-sequence learning with Transformer

### Positional Embedding
Layer ini menggabungkan embedding kata/token dengan embedding posisi sehingga model transformer dapat memahami urutan kata dalam sebuah kalimat. `PositionalEmbedding` ini memastikan bahwa transformer tidak hanya tahu kata apa yang ada, tetapi juga urutan kata dalam kalimat. Tanpa komponen ini, transformer hanya akan melihat kata sebagai “tas kata” (bag-of-words) tanpa urutan.

In [26]:
class PositionalEmbedding(Layer):
    def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
        super().__init__(**kwargs)
        
        #Menerjemahkan token ID (angka) menjadi vektor berdimensi output_dim.
        self.token_embeddings = Embedding(
            input_dim=input_dim, output_dim=output_dim, mask_zero=True  # aktifkan masking otomatis!
        )
        
        #intermediate = self.getPositionEncoding(seq_len=input_dim,d=vocab_size,n=output_dim)
        #Posisi 0, 1, 2, ..., sequence_length-1 diberi embedding seperti token biasa.
        #Perbedaan: input ke sini adalah urutan posisi, bukan ID kata.
        
        self.position_embeddings = Embedding(
            input_dim=sequence_length, output_dim=output_dim
        )
        self.sequence_length = sequence_length
        self.input_dim = input_dim
        self.output_dim = output_dim

    def call(self, inputs):
        # Gunakan panjang tetap dari self.sequence_length
        positions = tf.range(start=0, limit=tf.shape(inputs)[-1], delta=1)
        positions = self.position_embeddings(positions) # (1, sequence_length)
        x = self.token_embeddings(inputs)
        return x + positions

    def get_config(self):
        config = super().get_config()
        config.update({
            "sequence_length": self.sequence_length,
            "input_dim": self.input_dim,
            "output_dim": self.output_dim,
        })
        return config

### Transformer Encoder
`TransformerEncoder` ini adalah blok encoder dari arsitektur transformer, yang berfungsi untuk menangkap hubungan antar kata dalam sebuah kalimat dengan menggunakan self-attention dan feed-forward network. 

`TransformerEncoder = Self-Attention + Feed-Forward + Residual + Normalization`\
Fungsinya adalah membangun representasi kata yang kontekstual → setiap kata tidak berdiri sendiri, tapi tahu hubungannya dengan kata lain dalam kalimat.

In [27]:
class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads):
        super().__init__()
        self.attention = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential([
            layers.Dense(dense_dim, activation="relu"),
            layers.Dense(embed_dim),
        ])
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()

    def call(self, inputs, mask=None):
        attention_output = self.attention(inputs, inputs, attention_mask=None)
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)

### Transformer Decoder
`TransformerDecoder` bertugas menghasilkan urutan keluaran (output sequence) berdasarkan representasi yang diberikan encoder, sambil memastikan proses generasi bersifat autoregressive (hanya melihat token sebelumnya). 

Singkatnya, decoder bertugas menggabungkan dua sisi:
* Autoregressive generation (hanya melihat masa lalu lewat masked self-attention).
* Konteks global dari input (melalui cross-attention ke encoder).

In [28]:
class TransformerDecoder(layers.Layer):
    # Konstruktor yang berisi multihead ditambah masked self attention
    def __init__(self, embed_dim, dense_dim, num_heads):
        super().__init__()
        
        #Masked self-attention: hanya melihat ke token sebelumnya, agar tidak "mengintip masa depan".
        self.attention_1 = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        
        #Cross-attention: menghubungkan output dari encoder ke decoder (input dari bahasa sumber).
        self.attention_2 = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        
        #Feedforward network: 2 layer Dense sebagai pemrosesan non-linear.
        self.dense_proj = keras.Sequential([
            layers.Dense(dense_dim, activation="relu"),
            layers.Dense(embed_dim),
        ])
        
        #Normalisasi di setiap tahap (setelah residual connection).
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()

    #Membuat mask segitiga bawah (causal mask) berukuran [batch, seq_len, seq_len]
    #Mencegah posisi ke-i melihat ke token ke-j > i.
    #Digunakan agar saat pelatihan model decoder tidak tahu token setelahnya (future blindness).
    def get_causal_attention_mask(self, inputs):
        i = tf.range(tf.shape(inputs)[1])[:, tf.newaxis]
        j = tf.range(tf.shape(inputs)[1])
        mask = tf.cast(i >= j, dtype="int32")
        return tf.reshape(mask, (1, tf.shape(inputs)[1], tf.shape(inputs)[1]))

    def call(self, inputs, encoder_outputs, mask=None):
        
        #Buat Mask untuk self-attention, agar tidak “mengintip ke depan”.
        causal_mask = self.get_causal_attention_mask(inputs)
        
        #Masked Self-Attention
        #Fokus hanya ke token sebelumnya atau saat ini. Tambah residual + layernorm
        attention_output_1 = self.attention_1(inputs, inputs, attention_mask=causal_mask)
        attention_output_1 = self.layernorm_1(inputs + attention_output_1)
        
        #Cross Attention (Decoder attends to Encoder)
        #Menghubungkan representasi target dengan representasi yang
        #dihasilkan oleh encoder (misalnya dari kalimat Inggris).
        attention_output_2 = self.attention_2(attention_output_1, encoder_outputs, encoder_outputs)
        attention_output_2 = self.layernorm_2(attention_output_1 + attention_output_2)
        
        #FFN memperkuat representasi token. Lalu dilakukan residual connection dan layer normalization.
        proj_output = self.dense_proj(attention_output_2)
        return self.layernorm_3(attention_output_2 + proj_output)

### Bangun Model Transformer

#### Menentukan ukuran vektor representasi tiap token

In [29]:
embed_dim = 256

#### Menentukan ukuran hidden layer dalam blok FFN di encoder dan decoder

In [30]:
dense_dim = 2048

#### Menentukan ukuran banyak parallel attention heads dalam attention layer

In [31]:
num_heads = 8

#### Mendefinisikan input untuk bahasa sumber (English) dalam bentuk urutan token ID, dengan panjang dinamis (None).

In [32]:
encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="english")

#### Representasi token English yang mempertimbangkan urutan/posisi kata.

In [33]:
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(encoder_inputs)

#### Representasi konteks penuh untuk seluruh kalimat input (English).

In [34]:
encoder_outputs = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)

#### Input berupa token ID untuk bahasa target (Spanish), digunakan saat training.

In [35]:
decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="spanish")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(decoder_inputs)
x = TransformerDecoder(embed_dim, dense_dim, num_heads)(x, encoder_outputs)
x = layers.Dropout(0.5)(x)
decoder_outputs = layers.Dense(vocab_size, activation="softmax")(x)

In [36]:
transformer = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [37]:
transformer.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ english             │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ spanish             │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ positional_embeddi… │ (None, None, 256) │  3,845,120 │ english[0][0]     │
│ (PositionalEmbeddi… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ positional_embeddi… │ (None, None, 256) │  3,845,120 │ spanish[0][0]     │
│ (PositionalEmbeddi… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ transformer_encoder │ (None, None, 256) │  3,155,456 │ positional_embed… │
│ (TransformerEncode… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ transformer_decoder │ (None, None, 256) │  5,259,520 │ positional_embed… │
│ (TransformerDecode… │                   │            │ transformer_enco… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_3 (Dropout) │ (None, None, 256) │          0 │ transformer_deco… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, None,      │  3,855,000 │ dropout_3[0][0]   │
│                     │ 15000)            │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 19,960,216 (76.14 MB)

 Trainable params: 19,960,216 (76.14 MB)

 Non-trainable params: 0 (0.00 B)

#### Interpretasi Model
1. Input Layer
    * english (InputLayer) → (None, None)\
      Input urutan teks bahasa Inggris (panjang urutan fleksibel).
    * spanish (InputLayer) → (None, None)\
      Input urutan teks bahasa Spanyol (target sequence).
2. Positional Embedding
    * positional_embedding (english) → (None, None, 256) | 3,845,120 params
    * positional_embedding (spanish) → (None, None, 256) | 3,845,120 params
    * Setiap token (baik input maupun target) diubah menjadi vektor embedding berdimensi 256, lalu ditambahkan informasi posisi agar model tahu urutan kata.
    * Parameter = vocab_size * embedding_dim (di sini sekitar 15k * 256 = 3.8 juta).
3. Transformer Encoder
    * transformer_encoder → (None, None, 256) | 3,155,456 params
    * Menerima input embedding dari bahasa Inggris, lalu memprosesnya lewat:
        - Multi-Head Self-Attention
        - Feedforward Network (dense projection)
        - Residual + Normalisasi
    * Outputnya representasi kontekstual bahasa Inggris.
4. Transformer Decoder
    * transformer_decoder → (None, None, 256) | 5,259,520 params
    * Menerima:
      - Target sequence embedding (bahasa Spanyol)
      - Representasi encoder (bahasa Inggris)
    * Prosesnya meliputi:
      - Masked Self-Attention (hanya boleh lihat token sebelumnya di output).
      - Cross-Attention (menghubungkan target ke representasi encoder).
      - Feedforward Network.
    * Outputnya representasi token target yang sudah “tahu konteks input + output sebelumnya”.
5. Dropout
    * dropout_3 → (None, None, 256)\
      Lapisan regularisasi untuk mencegah overfitting.
6. Dense (Output Layer)
    * dense_4 (Dense) → (None, None, 15000) | 3,855,000 params
    * Layer linear yang memetakan dimensi 256 ke ukuran kosakata target (15k).
    * Setiap token target diproyeksikan ke distribusi probabilitas kata lewat softmax.
7. Parameter Total
    * Total params = ~20 juta (76 MB)
    * Semua trainable (tidak ada parameter beku).
8. Inti Arsitektur
   * Model ini adalah seq2seq Transformer untuk machine translation (English → Spanish) dengan:
     - Embedding dimensi 256
     - Encoder + Decoder stack
     - Ukuran kosakata target 15k
     - Total 20 juta parameter

In [38]:
#from tensorflow.keras.utils import plot_model
#from IPython.display import Image

#plot_model(transformer, to_file='transformer.png', show_shapes=True)
#Image(filename='transformer.png')

Setting lebih ringan (cepat latihan) akan diberi nama vanguard dan setting yang awal diberi nama transformer

## Bangun Model Vanguard

### Menentukan konfigurasi awal

In [39]:
embed_dim = 128
dense_dim = 512
num_heads = 4
sequence_length = 20
batch_size = 32

### Mendefinisikan input untuk bahasa sumber (English) dalam bentuk urutan token ID, dengan panjang dinamis (None).

In [40]:
encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="english")

### Representasi token English yang mempertimbangkan urutan/posisi kata.

In [41]:
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(encoder_inputs)

### Representasi konteks penuh untuk seluruh kalimat input (English).

In [42]:
encoder_outputs = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)

### Input berupa token ID untuk bahasa target (Spanish), digunakan saat training.

In [43]:
decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="spanish")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(decoder_inputs)
x = TransformerDecoder(embed_dim, dense_dim, num_heads)(x, encoder_outputs)
x = layers.Dropout(0.5)(x)
decoder_outputs = layers.Dense(vocab_size, activation="softmax")(x)

In [44]:
vanguard = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [45]:
vanguard.summary()

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ english             │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ spanish             │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ positional_embeddi… │ (None, None, 128) │  1,922,560 │ english[0][0]     │
│ (PositionalEmbeddi… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ positional_embeddi… │ (None, None, 128) │  1,922,560 │ spanish[0][0]     │
│ (PositionalEmbeddi… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ transformer_encode… │ (None, None, 128) │    396,032 │ positional_embed… │
│ (TransformerEncode… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ transformer_decode… │ (None, None, 128) │    660,096 │ positional_embed… │
│ (TransformerDecode… │                   │            │ transformer_enco… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_7 (Dropout) │ (None, None, 128) │          0 │ transformer_deco… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_9 (Dense)     │ (None, None,      │  1,935,000 │ dropout_7[0][0]   │
│                     │ 15000)            │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 6,836,248 (26.08 MB)

 Trainable params: 6,836,248 (26.08 MB)

 Non-trainable params: 0 (0.00 B)

### Interpretasi Model
1. Input Layer
    * english (InputLayer): menerima input berupa token urutan bahasa Inggris.
    * spanish (InputLayer): menerima input berupa token urutan bahasa Spanyol.
    * Keduanya berbentuk (None, None) → artinya panjang sequence fleksibel, batch size juga fleksibel.
2. Positional Embedding
    * english → positional_embedding:
      - Token embedding + positional embedding, menghasilkan representasi dimensi 128.
      - Jumlah parameter: 1,922,560
      - → ini berasal dari ukuran kosakata (input_dim) × output_dim (128).
    * spanish → positional_embedding_1:
      - Sama dengan di atas, untuk bahasa Spanyol.
      - Parameternya juga 1,922,560.
      - Dua embedding ini memastikan bahwa baik token Inggris maupun Spanyol punya representasi vektor + informasi posisi dalam sequence.
3. Transformer Encoder
    * Input: embedding dari bahasa Inggris.
    * Proses: self-attention + feedforward network + residual + layer norm.
    * Output: representasi kontekstual Inggris dalam dimensi 128.
    * Parameter: 396,032 (cukup ringan karena embed_dim hanya 128 dan jumlah head terbatas).
4. Transformer Decoder
    * Input utama: embedding Spanyol.
    * Input tambahan: hasil dari encoder (bahasa Inggris).
    * Proses: masked self-attention untuk decoder, cross-attention ke encoder output, lalu feedforward network.
    * Output: representasi bahasa Spanyol yang sudah terhubung dengan konteks bahasa Inggris.
    * Parameter: 660,096 (lebih besar daripada encoder, karena ada cross-attention tambahan).
5. Dropout
    * dropout_7: menjaga generalisasi, mencegah overfitting.
6. Dense Output Layer
    * dense_9 (Dense, 15000 units):
    * Output berupa distribusi probabilitas ke 15.000 kata (asumsi vocab size target = 15k).
    * Param: 1,935,000 (128 × 15000 + bias).
7. Total Parameter
    * 6,836,248 (~6.8M params)
    * Sangat ringan dibanding model Transformer standar (misalnya original Transformer bisa ratusan juta parameter).
    * Masuk akal disebut lebih “vanguard” karena compact, mudah dilatih dengan resource terbatas.
8. Interpretasi Ringkas
    * Model vanguard ini adalah seq2seq Transformer untuk terjemahan Inggris → Spanyol.
    * Bagian encoder memahami kalimat Inggris.
    * Bagian decoder menghasilkan kalimat Spanyol dengan memperhatikan hasil encoder.
    * Ukuran embedding yang kecil (128) dan vocab terbatas (15k) membuat model ini ringan, dengan hanya 6.8 juta parameter.
    * Dengan ini, model cocok untuk eksperimen akademik, portofolio, atau pelatihan di laptop tanpa GPU besar, sambil tetap mempertahankan arsitektur khas Transformer (encoder–decoder).

In [46]:
#from tensorflow.keras.utils import plot_model
#plot_model(vanguard, to_file='vanguard.png', show_shapes=True)
#from IPython.display import Image
#Image("vanguard.png")

## Compile dan Training

#### Transformer model

In [47]:
transformer.compile(
    optimizer="rmsprop",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

checkpoint = keras.callbacks.ModelCheckpoint(
    filepath="language_translation_checkpoint.weights.h5",
    save_weights_only=True,
    verbose=1,
    monitor="val_accuracy"
)

transformer.fit(train_ds, epochs=5, validation_data=val_ds, callbacks=[checkpoint])

Epoch 1/5
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.7119 - loss: 2.2096   
Epoch 1: saving model to language_translation_checkpoint.weights.h5
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 1919s 1s/step - accuracy: 0.7410 - loss: 1.7738 - val_accuracy: 0.8032 - val_loss: 1.2309
Epoch 2/5
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.8128 - loss: 1.1956   
Epoch 2: saving model to language_translation_checkpoint.weights.h5
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 1915s 1s/step - accuracy: 0.8258 - loss: 1.1047 - val_accuracy: 0.8527 - val_loss: 0.8826
Epoch 3/5
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.8513 - loss: 0.9121   
Epoch 3: saving model to language_translation_checkpoint.weights.h5
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 1909s 1s/step - accuracy: 0.8558 - loss: 0.8783 - val_accuracy: 0.8664 - val_loss: 0.7800
Epoch 4/5
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.8673 - loss: 0.7910   
Epoch 4: saving model to language_translation_checkpoint.weights.h5
1302/1302

#### Vanguard model

In [48]:
vanguard.compile(
    optimizer="rmsprop",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

checkpoint = keras.callbacks.ModelCheckpoint(
    filepath="language_translation_checkpoint.weights.h5",
    save_weights_only=True,
    verbose=1,
    monitor="val_accuracy"
)

vanguard.fit(train_ds, epochs=5, validation_data=val_ds, callbacks=[checkpoint])

Epoch 1/5
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 0s 442ms/step - accuracy: 0.7038 - loss: 2.5846  
Epoch 1: saving model to language_translation_checkpoint.weights.h5
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 636s 475ms/step - accuracy: 0.7359 - loss: 1.9178 - val_accuracy: 0.7923 - val_loss: 1.3261
Epoch 2/5
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 0s 441ms/step - accuracy: 0.7992 - loss: 1.3207  
Epoch 2: saving model to language_translation_checkpoint.weights.h5
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 615s 472ms/step - accuracy: 0.8105 - loss: 1.2402 - val_accuracy: 0.8381 - val_loss: 1.0146
Epoch 3/5
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 0s 442ms/step - accuracy: 0.8354 - loss: 1.0641  
Epoch 3: saving model to language_translation_checkpoint.weights.h5
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 616s 473ms/step - accuracy: 0.8403 - loss: 1.0310 - val_accuracy: 0.8561 - val_loss: 0.8999
Epoch 4/5
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 0s 441ms/step - accuracy: 0.8521 - loss: 0.9484  
Epoch 4: saving model to language_translation_checkpoint.weight

#### Perbandingan Performa Pelatihan
1. Awal Training (Epoch 1)
    * Transformer: train acc 74%, val acc 80% (cukup tinggi sejak awal → cepat adaptasi).
    * Vanguard: train acc 73%, val acc 79% (sedikit lebih rendah, tapi tetap bagus).
2. Perkembangan Selama 5 Epoch
    * Transformer: naik konsisten hingga val acc 87,7% dengan val loss 0.71.
    * Vanguard: juga naik stabil tapi sedikit di bawah, val acc 86,6% dengan val loss 0.83.
    * Transformer unggul sekitar +1% akurasi validasi dan lebih rendah val loss → lebih baik generalisasi.
3. Kecepatan Training
   * Transformer: 1900 detik/epoch (32 menit).
   * Vanguard: 616 detik/epoch (10 menit).
   * Vanguard 3x lebih cepat, tapi dengan trade-off akurasi sedikit lebih rendah.
#### Interpretasi
* Transformer memang lebih berat (kompleksitas self-attention, embedding lebih dalam, dll.), tapi memberi hasil lebih akurat dan stabil.
* Vanguard lebih ringan dan cepat dilatih, cocok jika sumber daya terbatas, namun ada kompromi pada akurasi akhir.
* Gap akurasi tidak terlalu besar (87,7% vs 86,6%), sehingga pilihan tergantung konteks penggunaan:
    - Jika akurasi absolut sangat penting (misalnya machine translation production level) → pilih Transformer.
    - Jika waktu dan resource jadi batasan (misalnya untuk prototyping, edge device, atau iterasi cepat) → Vanguard lebih efisien.
* Hasil ini menunjukkan keunggulan Transformer bukan hanya sebagai model kuat untuk bahasa alami, tapi juga memberikan baseline yang sangat solid.
* Vanguard sebagai varian yang lebih ringan bisa diposisikan sebagai alternatif trade-off antara akurasi dan kecepatan.

## Evaluate BLEU Score

#### Transformer model

In [52]:
# Asumsikan test_pairs = [(eng, spa), ...]
def decode_sequence(input_sentence):
    tokenized_input = source_vectorization([input_sentence])
    decoded_sentence = "[start]"
    for i in range(sequence_length):
        tokenized_target = target_vectorization([decoded_sentence])[:, :-1]
        predictions = transformer.predict({"english": tokenized_input, "spanish": tokenized_target}, verbose=0)
        sampled_token_index = np.argmax(predictions[0, i, :])
        sampled_token = target_vectorization.get_vocabulary()[sampled_token_index]
        if sampled_token == "[end]":
            break
        decoded_sentence += " " + sampled_token
    return decoded_sentence.replace("[start] ", "")

# BLEU Evaluation
for _ in range(5):
    input_text, target_text = random.choice(test_pairs)
    prediction = decode_sequence(input_text)
    reference = [target_text.replace("[start] ", "").replace(" [end]", "").split()]
    candidate = prediction.split()
    bleu_score = sentence_bleu(reference, candidate)
    print(f"Input: {input_text}\nPrediction: {prediction}\nTarget: {reference}\nBLEU: {bleu_score:.4f}\n")

### Save Final Model
transformer.save("transformer_translation_model.keras")

Input: Take it easy!
Prediction: toma fácil end                 
Target: [['¡Relajate!']]
BLEU: 0.0000

Input: Will he succeed or fail?
Prediction: va a triunfar o no end              
Target: [['¿Él', 'triunfará', 'o', 'fracasará?']]
BLEU: 0.0000

Input: I resign.
Prediction: yo [UNK] end                 
Target: [['Dimito.']]
BLEU: 0.0000

Input: He had more than enough money.
Prediction: Él tuvo más de dinero suficiente dinero end            
Target: [['Él', 'tenía', 'más', 'que', 'suficiente', 'dinero.']]
BLEU: 0.0000

Input: I was trying not to look.
Prediction: no estaba intentando mirar end               
Target: [['Trataba', 'de', 'no', 'mirar.']]
BLEU: 0.0000



#### Vanguard model

In [53]:
# Assume test_pairs = [(eng, spa), ...]
def decode_sequence(input_sentence):
    tokenized_input = source_vectorization([input_sentence])
    decoded_sentence = "[start]"
    for i in range(sequence_length):
        tokenized_target = target_vectorization([decoded_sentence])[:, :-1]
        predictions = vanguard.predict({"english": tokenized_input, "spanish": tokenized_target}, verbose=0)
        sampled_token_index = np.argmax(predictions[0, i, :])
        sampled_token = target_vectorization.get_vocabulary()[sampled_token_index]
        if sampled_token == "[end]":
            break
        decoded_sentence += " " + sampled_token
    return decoded_sentence.replace("[start] ", "")

# BLEU Evaluation
for _ in range(5):
    input_text, target_text = random.choice(test_pairs)
    prediction = decode_sequence(input_text)
    reference = [target_text.replace("[start] ", "").replace(" [end]", "").split()]
    candidate = prediction.split()
    bleu_score = sentence_bleu(reference, candidate)
    print(f"Input: {input_text}\nPrediction: {prediction}\nTarget: {reference}\nBLEU: {bleu_score:.4f}\n")

### Save Final Model
vanguard.save("vanguard_translation_model.keras")

Input: I hope we meet again someday soon.
Prediction: espero que nos [UNK] otra vez end             
Target: [['Espero', 'que', 'algún', 'día', 'pronto', 'nos', 'volvamos', 'a', 'ver.']]
BLEU: 0.0000

Input: Please put a lot of cream in my coffee.
Prediction: por favor [UNK] mucho en el café end            
Target: [['Ponele', 'mucha', 'crema', 'a', 'mi', 'café,', 'por', 'favor.']]
BLEU: 0.0000

Input: Come on, try again.
Prediction: ven a intentar end                
Target: [['Vamos,', 'inténtalo', 'otra', 'vez.']]
BLEU: 0.0000

Input: All human beings are mortal.
Prediction: todas las [UNK] son [UNK] end              
Target: [['Todos', 'los', 'humanos', 'son', 'mortales.']]
BLEU: 0.0000

Input: Can I have some of these?
Prediction: puedo tener algo de estas end              
Target: [['¿Puede', 'darme', 'algunos', 'de', 'éstos?']]
BLEU: 0.0000



OSError: [Errno 28] No space left on device

### Menyimpan arsitektur model dalam file json

In [51]:
model_json = transformer.to_json()
with open("translator.json", "w") as json_file:
    json_file.write(model_json)

**Menerjemahkan kalimat baru dengan model Transformer**

In [54]:
spa_vocab = target_vectorization.get_vocabulary()
spa_index_lookup = dict(zip(range(len(spa_vocab)), spa_vocab))
max_decoded_sentence_length = 20

**Menerjemahkan kalimat baru dengan model vanguard**

In [55]:
spa_vocab = target_vectorization.get_vocabulary()
spa_index_lookup = dict(zip(range(len(spa_vocab)), spa_vocab))
max_decoded_sentence_length = 20

### Output Testing and Decoding the output sequence with transformers

In [56]:
def transformer_decode_sequence(input_sentence):
    tokenized_input_sentence = source_vectorization([input_sentence])
    transformer_decoded_sentence = "[start]"
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = target_vectorization(
            [transformer_decoded_sentence])[:, :-1]
        predictions = transformer(
            [tokenized_input_sentence, tokenized_target_sentence])
        sampled_token_index = np.argmax(predictions[0, i, :])
        sampled_token = spa_index_lookup[sampled_token_index]
        transformer_decoded_sentence += " " + sampled_token
        if sampled_token == "[end]":
            break
    return transformer_decoded_sentence

### Output Testing and Decoding the output sequence with vanguard

In [57]:
def vanguard_decode_sequence(input_sentence):
    tokenized_input_sentence = source_vectorization([input_sentence])
    vanguard_decoded_sentence = "[start]"
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = target_vectorization(
            [vanguard_decoded_sentence])[:, :-1]
        predictions = vanguard(
            [tokenized_input_sentence, tokenized_target_sentence])
        sampled_token_index = np.argmax(predictions[0, i, :])
        sampled_token = spa_index_lookup[sampled_token_index]
        vanguard_decoded_sentence += " " + sampled_token
        if sampled_token == "[end]":
            break
    return vanguard_decoded_sentence

### Transformer translating output

In [58]:
test_eng_texts = [pair[0] for pair in test_pairs]
for _ in range(5):
    input_sentence = random.choice(test_eng_texts)
    print("-")
    print(input_sentence)
    print(transformer_decode_sequence(input_sentence))

-
Does prison reform criminals?
[start] [UNK] la cárcel end                
-
This juice would be even better with two ice cubes.
[start] este jugo sería mejor que dos con dos hielo end          
-
Show me your passport, please.
[start] muéstrame su pasaporte por favor end              
-
You have three seconds to make your choice.
[start] tienes tres semanas end                
-
What he says is true.
[start] lo que dice es cierto end              


### Vanguard translating output

In [59]:
test_eng_texts = [pair[0] for pair in test_pairs]
for _ in range(5):
    input_sentence = random.choice(test_eng_texts)
    print("-")
    print(input_sentence)
    print(vanguard_decode_sequence(input_sentence))

-
I can't decide which car to buy.
[start] no puedo decidir qué auto end              
-
You're quite smart.
[start] eres bastante inteligente end                
-
Tom is seeking a job.
[start] tom está buscando trabajo end               
-
She was heard to criticize the manager.
[start] ella estaba oído [UNK] a el [UNK] end            
-
Tom is extremely busy today.
[start] tom está muy ocupado hoy end              


## Evaluation using the BLEU score

### Transformer BLEU Score 

In [60]:
test_eng_texts = [pair[0] for pair in test_pairs]
test_spa_texts = [pair[1] for pair in test_pairs]
transformer_score = 0
transformer_bleu  = 0
for i in range(20):
    candidate = decode_sequence(test_eng_texts[i])
    reference = test_spa_texts[i].lower()
    print(candidate,reference)
    transformer_score = sentence_bleu(reference, candidate, weights=(1, 0, 0, 0))
    transformer_bleu+=transformer_score
    print(f"Transformer Score:{transformer_score}")
print(f"\nTransformer BLEU score : {round(transformer_bleu,2)}/20")

fui a la tienda a comprar algo de comprar un [UNK] y [UNK] end       [start] fui a la tienda a comprar champú y dentífrico. [end]
Transformer Score:0.2647058823529412
dónde está un hospital end                [start] ¿dónde hay un hospital? [end]
Transformer Score:0.34146341463414637
es hora de ir a la escuela end             [start] es hora de ir al colegio. [end]
Transformer Score:0.2857142857142857
no trabajo el domingo end                [start] no trabajo el domingo. [end]
Transformer Score:0.3499999999999999
queremos información end                  [start] queremos información. [end]
Transformer Score:0.3658536585365854
tom debería estar bien el lunes end              [start] tom debería estar bien para el lunes. [end]
Transformer Score:0.3125
te [UNK] este formulario por favor end              [start] ¿podría cumplimentar este formulario, por favor? [end]
Transformer Score:0.35294117647058826
todos los gato se ama a mi gato end            [start] todo el mundo quiere a mi gato.

In [61]:
print(f"Transformer BLEU score : {round(transformer_bleu,2)}/20")

Transformer BLEU score : 5.9/20


### Vanguard BLEU Score 

In [62]:
test_eng_texts = [pair[0] for pair in test_pairs]
test_spa_texts = [pair[1] for pair in test_pairs]
vanguard_score = 0
vanguard_bleu  = 0
for i in range(20):
    candidate = decode_sequence(test_eng_texts[i])
    reference = test_spa_texts[i].lower()
    print(candidate,reference)
    vanguard_score = sentence_bleu(reference, candidate, weights=(1, 0, 0, 0))
    vanguard_bleu+=vanguard_score
    print(f"Vanguard Score:{vanguard_score}")
print(f"Vanguard BLEU score : {round(vanguard_bleu,2)}/20")

fui a la tienda a comprar algo de comprar un [UNK] y [UNK] end       [start] fui a la tienda a comprar champú y dentífrico. [end]
Vanguard Score:0.2647058823529412
dónde está un hospital end                [start] ¿dónde hay un hospital? [end]
Vanguard Score:0.34146341463414637
es hora de ir a la escuela end             [start] es hora de ir al colegio. [end]
Vanguard Score:0.2857142857142857
no trabajo el domingo end                [start] no trabajo el domingo. [end]
Vanguard Score:0.3499999999999999
queremos información end                  [start] queremos información. [end]
Vanguard Score:0.3658536585365854
tom debería estar bien el lunes end              [start] tom debería estar bien para el lunes. [end]
Vanguard Score:0.3125
te [UNK] este formulario por favor end              [start] ¿podría cumplimentar este formulario, por favor? [end]
Vanguard Score:0.35294117647058826
todos los gato se ama a mi gato end            [start] todo el mundo quiere a mi gato. [end]
Vanguard Score

In [63]:
print(f"Vanguard BLEU score : {round(vanguard_bleu,2)}/20")

Vanguard BLEU score : 5.9/20


### Translation

#### Translation with transformers

In [64]:
from nltk.translate.bleu_score import sentence_bleu

# Ambil sampel acak dari data validasi
for _ in range(5):
    input_text, target_text = random.choice(val_pairs)
    prediction = transformer_decode_sequence(input_text)

    # Format referensi dan prediksi
    reference = [target_text.replace("[start] ", "").replace(" [end]", "").split()]
    candidate = prediction.split()

    transformer_bleu = sentence_bleu(reference, candidate)

    print(f"ENGLISH   : {input_text}")
    print(f"PREDICTED : {prediction}")
    print(f"TARGET    : {' '.join(reference[0])}")
    print(f"BLEU SCORE: {transformer_bleu:.4f}\n")

ENGLISH   : She denied having taken part in the scheme.
PREDICTED : [start] ella negó haber tomado parte en el plan end           
TARGET    : Ella negó haber tomado parte en el plan.
BLEU SCORE: 0.5170

ENGLISH   : His memory never ceases to astonish me.
PREDICTED : [start] su memoria no me [UNK] a [UNK] end            
TARGET    : Su memoria me sorprende.
BLEU SCORE: 0.0000

ENGLISH   : No, I'm not a teacher. I'm only a student.
PREDICTED : [start] no soy profesor solo un profesor solo estudiante end           
TARGET    : No, no soy maestro. Soy solo un estudiante.
BLEU SCORE: 0.0000

ENGLISH   : I'll be glad to.
PREDICTED : [start] me haré feliz end                
TARGET    : Será un placer.
BLEU SCORE: 0.0000

ENGLISH   : He was wrong in thinking that she'd come to see him.
PREDICTED : [start] Él estaba equivocado en ese día en ver que lo ver end        
TARGET    : Él se equivocaba al pensar que ella vendría a verle.
BLEU SCORE: 0.0000



#### Translation with vanguard

In [65]:
from nltk.translate.bleu_score import sentence_bleu

# Ambil sampel acak dari data validasi
for _ in range(5):
    input_text, target_text = random.choice(val_pairs)
    prediction = vanguard_decode_sequence(input_text)

    # Format referensi dan prediksi
    reference = [target_text.replace("[start] ", "").replace(" [end]", "").split()]
    candidate = prediction.split()

    vanguard_bleu = sentence_bleu(reference, candidate)

    print(f"ENGLISH   : {input_text}")
    print(f"PREDICTED : {prediction}")
    print(f"TARGET    : {' '.join(reference[0])}")
    print(f"BLEU SCORE: {vanguard_bleu:.4f}\n")

ENGLISH   : Tom knows the man Mary came with.
PREDICTED : [start] tom sabe el hombre que vino con mary end           
TARGET    : Tom conoce al hombre con el que vino Mary.
BLEU SCORE: 0.0000

ENGLISH   : Come on in. The water's nice.
PREDICTED : [start] ven a el buen agua end              
TARGET    : Métete. El agua está rica.
BLEU SCORE: 0.0000

ENGLISH   : I don't know my father's annual income.
PREDICTED : [start] no sé mi padre [UNK] [UNK] end             
TARGET    : No conozco los ingresos anuales de mi padre.
BLEU SCORE: 0.0000

ENGLISH   : Get on your knees.
PREDICTED : [start] [UNK] las [UNK] end                
TARGET    : Arrodíllate.
BLEU SCORE: 0.0000

ENGLISH   : I have two dogs, three cats, and six chickens.
PREDICTED : [start] tengo dos perros tres gatos y seis [UNK] end           
TARGET    : Tengo dos perros, tres gatos y seis gallinas.
BLEU SCORE: 0.2778



In [68]:
from nltk.translate.bleu_score import corpus_bleu

# Siapkan list untuk references dan candidates
references = []
candidates = []

# Loop seluruh pasangan kalimat di test set
for input_text, target_text in test_pairs:
    prediction = transformer_decode_sequence(input_text)
    
    # Bersihkan target (hilangkan token start/end)
    reference = target_text.replace("[start] ", "").replace(" [end]", "").split()
    candidate = prediction.split()
    
    # Tambahkan ke list untuk corpus BLEU
    references.append([reference])   # penting: list of list
    candidates.append(candidate)

# Hitung corpus BLEU
transformer_bleu_score = corpus_bleu(references, candidates)
print(f"Corpus Transformer BLEU score: {transformer_bleu_score:.4f}")

Corpus Transformer BLEU score: 0.1030


In [69]:
from nltk.translate.bleu_score import corpus_bleu

# Siapkan list untuk references dan candidates
references = []
candidates = []

# Loop seluruh pasangan kalimat di test set
for input_text, target_text in test_pairs:
    prediction = vanguard_decode_sequence(input_text)
    
    # Bersihkan target (hilangkan token start/end)
    reference = target_text.replace("[start] ", "").replace(" [end]", "").split()
    candidate = prediction.split()
    
    # Tambahkan ke list untuk corpus BLEU
    references.append([reference])   # penting: list of list
    candidates.append(candidate)

# Hitung corpus BLEU
vanguard_bleu_score = corpus_bleu(references, candidates)
print(f"Corpus Vanguard BLEU score: {vanguard_bleu_score:.4f}")

Corpus Vanguard BLEU score: 0.0867


## Evaluation Model

In [67]:
transformer_loss, transformer_acc = transformer.evaluate(val_ds)
vanguard_loss, vanguard_acc = vanguard.evaluate(val_ds)
print("Transformer Evaluation: ")
print(f"Validation Accuracy: {transformer_acc:.4f}")
print(f"Validation Loss: {transformer_loss:.4f}")
print(f"Transformer BLEU score : {round(transformer_bleu_score,2)}/20")
print("\nVanguard Evaluation: ")
print(f"Validation Accuracy: {vanguard_acc:.4f}")
print(f"Validation Loss: {vanguard_loss:.4f}")
print(f"Vanguard BLEU score : {round(vanguard_bleu,2)}/20")

279/279 ━━━━━━━━━━━━━━━━━━━━ 75s 269ms/step - accuracy: 0.8776 - loss: 0.7145 
279/279 ━━━━━━━━━━━━━━━━━━━━ 44s 156ms/step - accuracy: 0.8666 - loss: 0.8336
Transformer Evaluation: 
Validation Accuracy: 0.8776
Validation Loss: 0.7145
Transformer BLEU score : 0.0/20

Vanguard Evaluation: 
Validation Accuracy: 0.8666
Validation Loss: 0.8336
Vanguard BLEU score : 0.28/20


In [70]:
transformer_loss, transformer_acc = transformer.evaluate(val_ds)
vanguard_loss, vanguard_acc = vanguard.evaluate(val_ds)
print("Transformer Evaluation: ")
print(f"Validation Accuracy: {transformer_acc:.4f}")
print(f"Validation Loss: {transformer_loss:.4f}")
print(f"Corpus Transformer BLEU score : {transformer_bleu_score:.4f}")
print("\nVanguard Evaluation: ")
print(f"Validation Accuracy: {vanguard_acc:.4f}")
print(f"Validation Loss: {vanguard_loss:.4f}")
print(f"Corpus Vanguard BLEU score : {vanguard_bleu_score:.4f}")

279/279 ━━━━━━━━━━━━━━━━━━━━ 77s 274ms/step - accuracy: 0.8776 - loss: 0.7145 
279/279 ━━━━━━━━━━━━━━━━━━━━ 42s 151ms/step - accuracy: 0.8666 - loss: 0.8336
Transformer Evaluation: 
Validation Accuracy: 0.8776
Validation Loss: 0.7145
Corpus Transformer BLEU score : 0.1030

Vanguard Evaluation: 
Validation Accuracy: 0.8666
Validation Loss: 0.8336
Corpus Vanguard BLEU score : 0.0867


### Interpretasi
1. Akurasi dan Loss
    * Transformer unggul dalam akurasi (87.76% vs 86.66%) dan loss lebih rendah (0.71 vs 0.83).
    * Artinya, dari sisi prediksi token per token, Transformer lebih baik dalam menebak kata yang benar.
2. Corpus BLEU
    * Nilai BLEU keduanya masih rendah (sekitar 0.1).
    * Ini umum terjadi ketika model belum benar-benar fasih membentuk kalimat yang sesuai dengan ground truth.
    * Namun Transformer memiliki skor BLEU lebih tinggi (0.1030 vs 0.0867), menunjukkan kualitas urutan kata dan kesesuaian n-gram lebih baik dibanding Vanguard.
3. Kesesuaian antar-metrik
    * Akurasi tinggi + BLEU rendah → menunjukkan bahwa meskipun banyak kata yang diprediksi benar, urutan atau struktur kalimatnya sering tidak sesuai.
    * Transformer unggul tipis di BLEU, artinya selain lebih tepat di level kata, ia juga sedikit lebih baik dalam menyusun kata menjadi kalimat yang mirip target.
4. Kesimpulan
    * Transformer konsisten lebih unggul dibanding Vanguard di semua metrik (akurasi, loss, dan corpus BLEU).
    * Nilai BLEU yang rendah (sekitar 0.1) menandakan model masih jauh dari kualitas natural dalam menyusun kalimat, meskipun akurasi token sudah relatif tinggi.
    * Ini memperlihatkan bahwa akurasi tidak cukup untuk menilai kualitas teks, dan corpus BLEU membantu menunjukkan kelemahan model dalam menghasilkan urutan kata yang koheren.

# Kesimpulan
Hasil evaluasi menunjukkan bahwa Transformer memiliki keunggulan dalam akurasi dan loss, menandakan ketepatan prediksi token yang lebih baik dibanding Vanguard. Namun, nilai Corpus BLEU yang relatif rendah pada keduanya mengindikasikan bahwa susunan kalimat masih jauh dari ideal, walaupun prediksi kata per kata cukup tepat. Artinya, kualitas sintaksis dan semantik kalimat belum sepenuhnya terjaga. Kondisi ini menekankan bahwa akurasi dan BLEU tidak boleh dilihat terpisah, melainkan saling melengkapi, akurasi untuk menilai prediksi token, BLEU untuk menilai kelancaran kalimat. Dengan perbaikan strategi decoding (misalnya beam search) dan fine-tuning yang lebih matang, skor BLEU dapat ditingkatkan, sehingga Transformer semakin mendekati kemampuannya sebagai pondasi utama model bahasa generatif modern.

# Thank You